In [10]:
import pandas as pd
import numpy as np


In [11]:
df = pd.read_csv('Groceries_dataset.csv')
df

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk
...,...,...,...
38760,4471,08-10-2014,sliced cheese
38761,2022,23-02-2014,candy
38762,1097,16-04-2014,cake bar
38763,1510,03-12-2014,fruit/vegetable juice


In [12]:
df.isnull().sum()

Member_number      0
Date               0
itemDescription    0
dtype: int64

In [13]:
df['itemDescription'].value_counts().head()

itemDescription
whole milk          2502
other vegetables    1898
rolls/buns          1716
soda                1514
yogurt              1334
Name: count, dtype: int64

In [14]:
member_shopping_frequency = df.groupby('Member_number')['Date'].count().sort_values(ascending=False)
print(member_shopping_frequency)

Member_number
3180    36
3737    33
3050    33
2051    33
3915    31
        ..
3533     2
2302     2
4824     2
1084     2
3377     2
Name: Date, Length: 3898, dtype: int64


In [15]:
from apyori import apriori

In [16]:
group_same_customer = df.sort_values(by='Member_number', ascending=True)
group_same_customer['itemDescription'].str.strip()

1629                    soda
13331             whole milk
8395              whole milk
4843                 sausage
17778     pickled vegetables
                ...         
34885    semi-finished bread
25489       other vegetables
9340            bottled beer
27877                 onions
3578                    soda
Name: itemDescription, Length: 38765, dtype: object

In [17]:
transactions_list = []
for _, group in group_same_customer.groupby('Member_number'):
    customer_purchases = group['itemDescription'].tolist()
    transactions_list.append(customer_purchases)
transactions_list

[['soda',
  'whole milk',
  'whole milk',
  'sausage',
  'pickled vegetables',
  'canned beer',
  'yogurt',
  'misc. beverages',
  'salty snack',
  'sausage',
  'semi-finished bread',
  'hygiene articles',
  'pastry'],
 ['whole milk',
  'sausage',
  'curd',
  'soda',
  'beef',
  'frankfurter',
  'white bread',
  'frankfurter',
  'whipped/sour cream',
  'rolls/buns',
  'whole milk',
  'soda'],
 ['other vegetables',
  'frozen vegetables',
  'specialty chocolate',
  'sugar',
  'butter',
  'whole milk',
  'tropical fruit',
  'butter milk'],
 ['rolls/buns',
  'dental care',
  'detergent',
  'sausage',
  'rolls/buns',
  'rolls/buns',
  'frozen meals',
  'root vegetables'],
 ['whole milk',
  'packaged fruit/vegetables',
  'whole milk',
  'canned beer',
  'chocolate',
  'dish cleaner',
  'shopping bags',
  'other vegetables',
  'pip fruit',
  'red/blush wine',
  'root vegetables',
  'whole milk',
  'frozen fish',
  'hygiene articles',
  'pastry',
  'chocolate',
  'rolls/buns',
  'other vegetab

In [18]:
association_rules = apriori(transactions_list, min_support=0.001, min_confidence=0.05, min_lift=4, min_length=2,
                            max_length=2)
rules_list = list(association_rules)
rules_list

[RelationRecord(items=frozenset({'soups', 'Instant food products'}), support=0.001026167265264238, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Instant food products'}), items_add=frozenset({'soups'}), confidence=0.06666666666666667, lift=5.413888888888889), OrderedStatistic(items_base=frozenset({'soups'}), items_add=frozenset({'Instant food products'}), confidence=0.08333333333333333, lift=5.413888888888889)]),
 RelationRecord(items=frozenset({'canned fruit', 'soft cheese'}), support=0.001026167265264238, ordered_statistics=[OrderedStatistic(items_base=frozenset({'canned fruit'}), items_add=frozenset({'soft cheese'}), confidence=0.19047619047619047, lift=5.050858438613541)]),
 RelationRecord(items=frozenset({'canned vegetables', 'rum'}), support=0.001026167265264238, ordered_statistics=[OrderedStatistic(items_base=frozenset({'rum'}), items_add=frozenset({'canned vegetables'}), confidence=0.125, lift=6.090625)]),
 RelationRecord(items=frozenset({'tea', 'cat food'}), supp

In [19]:
len(rules_list)

18

In [20]:
def print_rules(rules):
    for rule in rules:
        print('rule.items=', list(rule.items))
        print('rule.support=', rule.support)

        for os in rule.ordered_statistics:
            print('\titems_base=', list(os.items_base))
            print('\tlifted_item =', list(os.items_add))
            print('\tlift=', os.lift)
            print('\tconfidence (i.e. cond prob {} if {})='.format(list(os.items_add), list(os.items_base)),
                  os.confidence)
            print('\n')


print_rules(rules_list)

rule.items= ['soups', 'Instant food products']
rule.support= 0.001026167265264238
	items_base= ['Instant food products']
	lifted_item = ['soups']
	lift= 5.413888888888889
	confidence (i.e. cond prob ['soups'] if ['Instant food products'])= 0.06666666666666667


	items_base= ['soups']
	lifted_item = ['Instant food products']
	lift= 5.413888888888889
	confidence (i.e. cond prob ['Instant food products'] if ['soups'])= 0.08333333333333333


rule.items= ['canned fruit', 'soft cheese']
rule.support= 0.001026167265264238
	items_base= ['canned fruit']
	lifted_item = ['soft cheese']
	lift= 5.050858438613541
	confidence (i.e. cond prob ['soft cheese'] if ['canned fruit'])= 0.19047619047619047


rule.items= ['canned vegetables', 'rum']
rule.support= 0.001026167265264238
	items_base= ['rum']
	lifted_item = ['canned vegetables']
	lift= 6.090625
	confidence (i.e. cond prob ['canned vegetables'] if ['rum'])= 0.125


rule.items= ['tea', 'cat food']
rule.support= 0.0017957927142124167
	items_base= ['t

In [21]:

list_item = []
list_lifted_item = []
list_lift = []
for rule in rules_list:
    for os in rule.ordered_statistics:
        list_item.append(list(os.items_base))
        list_lifted_item.append(list(os.items_add))
        list_lift.append(os.lift)

list_item

[['Instant food products'],
 ['soups'],
 ['canned fruit'],
 ['rum'],
 ['tea'],
 ['ready soups'],
 ['cookware'],
 ['curd cheese'],
 ['mustard'],
 ['decalcifier'],
 ['dental care'],
 ['dog food'],
 ['frozen potato products'],
 ['female sanitary products'],
 ['hair spray'],
 ['light bulbs'],
 ['tea'],
 ['jam'],
 ['rice'],
 ['liver loaf'],
 ['photo/film'],
 ['ready soups'],
 ['soups']]

In [25]:
rules_df = pd.DataFrame(data={"Item": list_item, "Add Item": list_lifted_item, "Lift": list_lift})
rules_df

,Item,Add Item,Lift
0,[Instant food products],[soups],5.413889
1,[soups],[Instant food products],5.413889
2,[canned fruit],[soft cheese],5.050858
3,[rum],[canned vegetables],6.090625
4,[tea],[cat food],5.909898
5,[ready soups],[chewing gum],5.973946
6,[cookware],[ice cream],4.168984
7,[curd cheese],[mustard],4.655996
8,[mustard],[curd cheese],4.655996
9,[decalcifier],[dessert],5.140785


In [26]:
rules_df.sort_values("Lift", ascending=False).head(10)

,Item,Add Item,Lift
18,[rice],[jam],9.358944
17,[jam],[rice],9.358944
3,[rum],[canned vegetables],6.090625
21,[ready soups],[oil],5.987711
5,[ready soups],[chewing gum],5.973946
4,[tea],[cat food],5.909898
1,[soups],[Instant food products],5.413889
0,[Instant food products],[soups],5.413889
9,[decalcifier],[dessert],5.140785
2,[canned fruit],[soft cheese],5.050858
